### autoloader json to delta table 

In [0]:
from pyspark.sql.functions import *

# Step 1: Define the schema (recommended for Auto Loader)
from pyspark.sql.types import StructType, StringType, IntegerType

schema = StructType() \
    .add("id", IntegerType()) \
    .add("city", StringType()) \
    .add("country", StringType()) \
    .add("sales", IntegerType())

# Step 2: Read from folder using Auto Loader (cloudFiles)
df = (spark.readStream
      .format("cloudFiles")
      .option("cloudFiles.format", "json")  # Change to csv, parquet, etc. if needed
      .schema(schema)
      .load("/mnt/incoming-data/")
     )

# Step 3: Write to Delta table
(df.writeStream
   .format("delta")
   .outputMode("append")
   .option("checkpointLocation", "/mnt/incoming-data/_checkpoint")  # Required
   .start("/mnt/bronze/sales_city")  # Path where Delta table data is stored
)

In [0]:

# after stream starts create delta table because delta table needs to have data so when stream will start it will write data to checkpoints directory or create earlier delta table having data to upload incoming stream 

spark.sql("""
CREATE TABLE IF NOT EXISTS sales_city
USING DELTA
LOCATION '/mnt/bronze/sales_city'
""")